In [1]:
from datasets import load_dataset
import random
import pandas as pd
import datasets
from IPython.display import display,HTML
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer
import numpy as np
import evaluate

dataset= load_dataset("yelp_review_full")
print(f'yelo_review_full dataset loaded.')

tokenizer=AutoTokenizer.from_pretrained("bert-base-cased")
print(f'bert-base-cased pretrained  loaded.')

metric=evaluate.load("accuracy")
def tokenize_function(examples):
    return tokenizer(examples["text"],padding="max_length",truncation=True)
    
def compute_metrics(eval_pred):
    logits,labels=eval_pred
    predictions=np.argmax(logits,axis=-1)
    return metric.compute(predictions=predictions,references=labels)

tokenized_datasets=dataset.map(tokenize_function,batched=True)
print(f'dataset map function updated.')

small_train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset=tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
print(f'dataset 1000 items')

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",num_labels=5)
model_dir="./gen/models/bert-base-cased-finetune-yelp"


training_args=TrainingArguments(output_dir=model_dir,
                               evaluation_strategy="epoch",
                               per_device_train_batch_size=48,
                               num_train_epochs=3,
                               logging_steps=30)
print(f'training_args updated')

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)
print(f'trainer updated')

print(f'start to train')
trainer.train()
print(f'Train Completed!')
small_test_dataset=tokenized_datasets["test"].shuffle(seed=64).select(range(100))
print(trainer.evaluate(small_test_dataset))
print(f'Evaluate Completed!')
trainer.save_model(model_dir)
print(f'Save model Completed!')
trainer.save_state()
print(f'Save state Completed!')

2024-01-31 15:39:58.018958: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-31 15:39:58.711456: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 15:39:58.711542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 15:39:58.833064: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 15:39:59.081142: I tensorflow/core/platform/cpu_feature_guar

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

yelo_review_full dataset loaded.


...

/home/ubuntu/miniconda3/envs/transformers/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987288773/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


training_args updated
trainer updated
start to train


Epoch,Training Loss,Validation Loss
